In [16]:
!pip install telethon


[notice] A new release of pip is available: 23.1.2 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [17]:
from telethon import TelegramClient, sync
import pandas as pd
from datetime import timedelta

api_id = 27978349
api_hash = '174f6f06f7e1b241be23ccfe327f94a6'
client = TelegramClient('a', api_id, api_hash)
await client.start()
await client.connect()

In [19]:
channel = 'pikabu'
chats = await client.get_messages(channel, 1000)

In [21]:
prev_date = 0
for i in range(len(chats)):
    if 'MessageMediaDocument' in str(chats[i].media):
        continue
    if i != 0:
        if prev_date - chats[i].date < timedelta(seconds = 5):
            continue
    prev_date = chats[i].date
    await chats[i].download_media('/data/notebook_files/' + f'{channel}' + f'/{chats[i].date}')

In [22]:
message = []
time = []
for chat in chats:
    message.append(chat.message)
    time.append(chat.date)

In [23]:
import numpy as np
df = pd.DataFrame({'message': message, 'time': time, 'ad':[np.nan for i in range(len(message))]})

In [25]:
df.to_csv(f'/data/notebook_files/{channel}.csv', index = False)

In [27]:
to_dro = []
for i in range(len(df)):
    if type(df.message.iloc[i]) == float and ((len(df[df.time == df.time.iloc[i]]) > 1) or (len(df[df.time == df.time.iloc[i]+timedelta(seconds=1)]) > 0)or (len(df[df.time == df.time.iloc[i]-timedelta(seconds=1)]) > 0)):
        to_dro.append(i)

In [28]:
df = df.drop(index = to_dro)

In [29]:
df.to_csv(f'/data/notebook_files/{channel}.csv', index = False)

In [0]:
await client.disconnect()